In [5]:
"""import our data"""

import numpy as np
from data import get_data_from_csv, get_labels_and_data_1st_column

# DATA FROM HERE: https://pjreddie.com/projects/mnist-in-csv/
file_test = '../data/MNIST/mnist_test.csv'
file_train = '../data/MNIST/mnist_train.csv'

data_test, m_test, n_test = get_data_from_csv(file_test)
Y_test, X_test = get_labels_and_data_1st_column(data_test)

data_train, m_train, n_train = get_data_from_csv(file_train)
Y_train, X_train = get_labels_and_data_1st_column(data_train)

assert n_test == n_train
n = n_test
m = m_test + m_train



In [6]:
"""making sure that our Y_test/Y_train are actually labels"""

assert max(Y_test) == 9
assert max(Y_train) == 9
assert max(X_test[0]) != 9
assert max(X_train[0]) != 9

# display(Y_test[:100])
# display(Y_train[:100])
# display(X_test[500][:100])
# display(X_train[500][:100])

In [7]:
# normalize pixel data to values between 0 and 1. Max = 255, min = 0, thus divide by 255
X_train = X_train / 255.
X_test = X_test / 255.

In [51]:
"""
FORWARD PASS
Give X
A0 = X :: [784,1]
A1[10,1] = W1[10,784] * X[784,1] + b1[10,1]
Z1[10,1] = RelU(A1[10,1])
A2[10,1] = W2[10,10] * Z1[10,1] + b2[10,1]
Y_hat[10,1] = softmax(A2[10,1])
Receive Y_hat
"""


def initialize_w_b():
    W1 = np.random.randn(10, 784)
    b1 = np.random.randn(10,)
    W2 = np.random.randn(10,10)
    b1 = np.random.randn(10,)
    return W1, b1, W2, b1

def ReLU(n: float) -> float:
    """rectified linear unit activation function"""
    return np.maximum(n, 0)

def ReLU_deriv(n: float) -> int:
    """"derivative of ReLU"""
    return n > 0

def softmax(array: np.array) -> np.array:
    return np.exp(array) / sum(np.exp(array))

def forward_pass(X, W1, b1, W2, b2):
    A0 = X
    Z1 = W1.dot(A0) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    Y_hat = softmax(Z2)
    return Y_hat

def one_hot_encode(Y: np.array):
    # first instantiate 0's which should be an array of len(Y) max(Y) 
    one_hot = np.zeros((Y.size,np.max(Y)+1))
    one_hot[np.arange(Y.size), Y] = 1
    return one_hot


def backwards_propagation(Y_hat, Y, Z2, A1, Z1, W1, b1, W2, b2, m, X):
    """
    Y_hat = [.1, .1, .7, .1]
    Y = [0,0,1,0]
    dZ2 = [.1, .1, -.3, .1]

    L = Y - Y_hat
    dL/dY_hat = -1 
    dY_hat/dZ2 = 1
    
    dZ2 = dL/dZ2 = dL/dY_hat * dY_hat/dZ2 = (Y_hat - Y)     # * 1 / m
    
    dW2 = dL/dW2 = dL/dZ2 * dZ2/dW2 = dZ2 * 1 / m * A1
    db2 = dL/db2 = dL/dZ2 * dZ2/db2 = dZ2 * 1 / m 
    
    dA1 = dL/dA1 = dL/Z2 * dZ2/dA1 = dZ2 * W2
    
    dZ1 = dL/dZ1 = dL/dA1 * dA1/dZ1 = dA1 * ReLU_deriv(Z1)

    dW1 = dL/dW1 = dL/dZ1 * dZ1/dW1 = dZ1 * 1 / m  * X
    db1 = dL/db1 = dL/dZ1 * dZ1/db1 = dZ1 * 1 / m 


    CALCULATIONS
    dZ2[10,] = Y_hat[10,] - Y[10,]
    dW2[10,10] = 1/m * dZ2[10,1] * A1[10,1].T[1,10]
    dA1[10,1] = W2.T[10,10].dot(dZ2[10,1]) # something weird here
    dZ1[10,1] = dA1[10,1] * ReLU_deriv(Z1[10,1])[10,1]


    """
    dZ2 = Y_hat - Y
    
    dW2 = 1/m * dZ2.dot(A1.T)
    db2 = 1/m * np.sum(dZ2)/len(dZ2) # this is wrong

    dA1 = W2.T.dot(dZ2)
    dZ1 = dA1 * ReLU_deriv(Z1)

    dW1 = 1/m * dZ1.dot(X.T)
    db1 = 1/m * dZ1

    return dW1, db1, dW2, db2




def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, learning_rate):
    W1 = W1 - learning_rate*dW1
    b1 = b1 - learning_rate*db1
    W2 = W2 - learning_rate*dW2
    b2 = b2 - learning_rate*db2
    
    return W1, b1, W2, b1


In [52]:
Y = np.array([0,1,3,4,4,2])
one_hot = one_hot_encode(Y)
print(one_hot)

[[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0.]]


In [13]:
W1, b1, W2, b2 = initialize_w_b()

X = X_test[:,0]

forward_pass(X, W1, b1, W2, b2)


A0.shape=(784,)
A1.shape=(10,)
Z1.shape=(10,)
A2.shape=(10,)
Y_hat.shape=(10,)


array([2.19026912e-07, 1.23076400e-06, 8.14249733e-13, 1.34590696e-04,
       1.54466230e-17, 2.99065148e-10, 3.08124793e-06, 1.18054488e-09,
       7.92935750e-16, 9.99860877e-01])

In [ ]:
"""
a = [0, .5, 2, 1]

sums = e^0 + e^.5 + e^2 + e^1


softmax(a) = [e^0/sums, e^.5/sums, e^2/sums, e^1/sums]

"""